# Dask Array

__Автор задач: Блохин Н.В. (NVBlokhin@fa.ru)__

Материалы:
* Макрушин С.В. Лекция "Dask"
* https://docs.dask.org/en/latest/array.html

In [2]:
!pip install dask

     ---------------------------------------- 1.1/1.1 MB 313.1 kB/s eta 0:00:00
     ------------------------------------ 151.7/151.7 kB 312.2 kB/s eta 0:00:00
     -------------------------------------- 55.8/55.8 kB 224.6 kB/s eta 0:00:00


Error processing line 1 of C:\Users\icom1\AppData\Local\Programs\Python\Python310\lib\site-packages\matplotlib-3.5.3-py3.10-nspkg.pth:

  Traceback (most recent call last):
    File "C:\Users\icom1\AppData\Local\Programs\Python\Python310\lib\site.py", line 186, in addpackage
      exec(line)
    File "<string>", line 1, in <module>
    File "<frozen importlib._bootstrap>", line 568, in module_from_spec
  AttributeError: 'NoneType' object has no attribute 'loader'

Remainder of file ignored
Error processing line 1 of C:\Users\icom1\AppData\Local\Programs\Python\Python310\lib\site-packages\matplotlib-3.6.0-py3.10-nspkg.pth:

  Traceback (most recent call last):
    File "C:\Users\icom1\AppData\Local\Programs\Python\Python310\lib\site.py", line 186, in addpackage
      exec(line)
    File "<string>", line 1, in <module>
    File "<frozen importlib._bootstrap>", line 568, in module_from_spec
  AttributeError: 'NoneType' object has no attribute 'loader'

Remainder of file ignored

[notice] 

In [3]:
import dask.array as da
import h5py
import numpy as np
import dask
import pandas as pd

In [4]:
!pip install graphviz

     -------------------------------------- 47.0/47.0 kB 294.4 kB/s eta 0:00:00


Error processing line 1 of C:\Users\icom1\AppData\Local\Programs\Python\Python310\lib\site-packages\matplotlib-3.5.3-py3.10-nspkg.pth:

  Traceback (most recent call last):
    File "C:\Users\icom1\AppData\Local\Programs\Python\Python310\lib\site.py", line 186, in addpackage
      exec(line)
    File "<string>", line 1, in <module>
    File "<frozen importlib._bootstrap>", line 568, in module_from_spec
  AttributeError: 'NoneType' object has no attribute 'loader'

Remainder of file ignored
Error processing line 1 of C:\Users\icom1\AppData\Local\Programs\Python\Python310\lib\site-packages\matplotlib-3.6.0-py3.10-nspkg.pth:

  Traceback (most recent call last):
    File "C:\Users\icom1\AppData\Local\Programs\Python\Python310\lib\site.py", line 186, in addpackage
      exec(line)
    File "<string>", line 1, in <module>
    File "<frozen importlib._bootstrap>", line 568, in module_from_spec
  AttributeError: 'NoneType' object has no attribute 'loader'

Remainder of file ignored

[notice] 

In [6]:
dask.__version__

'2022.11.1'

## Задачи для совместного разбора

1. Создайте массив размерностью 1000 на 300000, заполненный числами из стандартного нормального распределения. Исследуйте основные характеристики полученного массива. Визуализируйте граф вычисления задачи.

In [11]:
path = 'C://Users//icom1//OneDrive - icom//Рабочий стол//TOBD_datasets//'

In [12]:
import numpy as np
import h5py
with h5py.File(path+'demo.h5', 'w') as hdf:
    hdf.create_dataset('arr', data=np.random.normal(0, 1, size=(1000, 300_000)))

In [14]:
#with h5py.File(path+'demo.h5', 'r') as hdf:
    # ...
    
#...
hdf = h5py.File(path+'demo.h5', 'r')

dset = hdf['arr']

dset

#need to close at the end

<HDF5 dataset "arr": shape (1000, 300000), type "<f8">

In [15]:
arr = da.from_array(dset)
arr
#arr = da.from_array(dset, chunks=(1000, 20000))

dask.array<array, shape=(1000, 300000), dtype=float64, chunksize=(1000, 16777), chunktype=numpy.ndarray>

In [16]:
type(arr)

dask.array.core.Array

In [22]:
arr.mean()

dask.array<mean_agg-aggregate, shape=(), dtype=float64, chunksize=(), chunktype=numpy.ndarray>

In [23]:
# arr.mean().visualize()

In [26]:
(arr.mean() + 5) / 2

dask.array<truediv, shape=(), dtype=float64, chunksize=(), chunktype=numpy.ndarray>

In [29]:
arr.mean().compute()

-2.904264701186744e-05

In [34]:
arr1 = arr / 2
m = arr1.mean()
s = arr1.sum()
m

dask.array<mean_agg-aggregate, shape=(), dtype=float64, chunksize=(), chunktype=numpy.ndarray>

In [32]:
%time s.compute(), m.compute()

CPU times: total: 7.84 s
Wall time: 3.44 s


(-4356.397051780116, -1.452132350593372e-05)

In [36]:
%%time

s_c, m_c = dask.compute(
    s,
    m
)
s_c, m_c

CPU times: total: 4.53 s
Wall time: 1.7 s


(-4356.397051780116, -1.452132350593372e-05)

2. Посчитайте сумму квадратов элементов массива, созданного в задаче 1. Создайте массив `np.array` такого же размера и сравните скорость решения задачи с использование `da.array` и `np.array`

In [38]:
%%time
arr_np = np.random.normal(0, 1, size=(1000, 300_000))
(arr_np ** 2).sum()

CPU times: total: 12.1 s
Wall time: 12.1 s


299974860.5584305

In [40]:
%%time
arr_da = da.random.normal(0, 1, size=(1000, 300_000))
(arr_da ** 2).sum().compute()

CPU times: total: 22 s
Wall time: 3.84 s


300023980.6052066

In [42]:
def f(arr):
    return arr + 2
f(arr_da) #.compute()

dask.array<add, shape=(1000, 300000), dtype=float64, chunksize=(1000, 16777), chunktype=numpy.ndarray>

In [46]:
%%time
arr1 = arr / 2
m = arr1.mean().persist()

CPU times: total: 3.67 s
Wall time: 1.62 s


In [47]:
m

dask.array<mean_agg-aggregate, shape=(), dtype=float64, chunksize=(), chunktype=numpy.ndarray>

In [48]:
%%time
m.compute()

CPU times: total: 0 ns
Wall time: 0 ns


-1.452132350593372e-05

In [49]:
%%time
arr1.mean().compute()

CPU times: total: 3.56 s
Wall time: 1.58 s


-1.452132350593372e-05

In [43]:
arr[:50, :50]

dask.array<getitem, shape=(50, 50), dtype=float64, chunksize=(50, 50), chunktype=numpy.ndarray>

In [44]:
arr[arr>0, :] #isn`t possible

NotImplementedError: Slicing with dask.array of bools only permitted when the indexer has only one dimension or when it has the same dimension as the sliced array

In [45]:
arr[arr>0]

C:\Users\icom1\AppData\Local\Programs\Python\Python310\lib\site-packages\dask\array\core.py:1971: PerformanceWarning: Reshaping is producing a large chunk. To accept the large
chunk and silence this warning, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    ...     array.reshape(shape)

To avoid creating the large chunks, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    ...     array.reshape(shape)Explictly passing ``limit`` to ``reshape`` will also silence this warning
    >>> array.reshape(shape, limit='128 MiB')
  self, index2 = slice_with_bool_dask_array(self, index2)


dask.array<getitem, shape=(nan,), dtype=float64, chunksize=(nan,), chunktype=numpy.ndarray>

## Лабораторная работа 11

__При решении данных задач не подразумевается использования циклов или генераторов Python в ходе работы с пакетами `numpy`, `pandas` и `dask`, если в задании не сказано обратного. Решения задач, в которых для обработки массивов `numpy`, структур `pandas` или структур `dask` используются явные циклы (без согласования с преподавателем), могут быть признаны некорректными и не засчитаны.__

In [1]:
path = 'C://Users//icom1//OneDrive - icom//Рабочий стол//TOBD_datasets//'
import dask.array as da
import h5py
import numpy as np
import dask
import pandas as pd

В ходе выполнения все операции вычислений (расчет средних значений, расчет косинусной близости и т.д.) проводятся над `dask.array` и средствами пакета `dask`, если в задании не сказано обратного. Переход от `dask.array` к `numpy.array` или `pd.DataFrame` возможен исключительно для демонстрации результата в конце решения задачи. Если в задаче используются результаты выполнения предыдущих задач, то подразумевается, что вы используете результаты в виде `dask.array` (то есть то, что было получено до вызова `compute`, а не после).

1\. Считайте датасет `embeddings` из файла `recipe_embeddings.h5` в виде `dask.array`. Выведите на экран основную информацию о массиве: размер, форму, тип, количество и размер сегментов. 

In [120]:
hdf = h5py.File(path+'recipe_embeddings.h5', 'r+')

dset = hdf['embeddings']

dset

<HDF5 dataset "embeddings": shape (1200000, 312), type "<f4">

In [121]:
arr = da.from_array(dset)
arr

dask.array<array, shape=(1200000, 312), dtype=float32, chunksize=(107546, 312), chunktype=numpy.ndarray>

2\. Посчитайте и выведите на экран среднее значение всех элементов массива. Исследуйте, как влияет значение аргумента `chunks` при создании `dask.array` на скорость выполнения операции поиска среднего. 

Пусть $M$ - количество строк в массиве, $N$ - количество столбцов в массиве, `chunks=(r,c)`. Сравните несколько вариантов:
* $r=M$, $с \ll N$ , 
* $r \ll M$, $c=N$ 
* $r = M$, $c = N$ 
* значения $r, c$ по умолчанию.

Выберите наиболее оптимальные значения $r$ и  $c$ в смысле скорости вычислений и далее продолжайте работу с ними.

In [4]:
%%time
arr = da.from_array(dset, chunks=(1200000, 3))
arr.mean().compute()

CPU times: total: 1min 6s
Wall time: 1min 11s


0.0023777562

In [5]:
%%time
arr = da.from_array(dset, chunks=(1200, 312))
arr.mean().compute()

CPU times: total: 2.66 s
Wall time: 1.68 s


0.0023777564

In [6]:
%%time
arr = da.from_array(dset, chunks=(1200000, 312))
arr.mean().compute()

CPU times: total: 1.25 s
Wall time: 1.26 s


0.0023777678

In [7]:
%%time
arr = da.from_array(dset)
arr.mean().compute()

CPU times: total: 1.52 s
Wall time: 951 ms


0.0023777557

Оптимально разбиение, которое предлагает сам Dask, поэтому будем использовать его P.S. разбиение на 3 столбца и 1_200_000 строк - трагедия
---

3\. Опишите пространство, в котором расположены эмбеддинги, посчитав минимальное и максимальное значение для каждой из координат. Сведите результаты в таблицу `pd.DataFrame`, состоящую из двух строк и 312 столбцов. Задайте индексы строк "min" и "max". Названия столбцов сделайте вида $x_i$. Выведите полученную таблицу на экран.

Решите задачу двумя способами. В первом варианте сделайте два вызова метода `compute` для расчета каждого из векторов максимальных и минимальных значений. Во втором варианте сделайте один вызов функции `dask.compute` для одновременного расчета двух векторов. Сравните время выполнения двух решений.

In [8]:
%%time
minimum = arr.min(axis=0)
maximum = arr.max(axis=0)
pd.DataFrame([minimum.compute(), maximum.compute()], columns=[f'x{i}' for i in range(arr.shape[1])])

CPU times: total: 2.34 s
Wall time: 1.64 s


,x0,x1,x2,x3,x4,x5,x6,x7,x8,x9,...,x302,x303,x304,x305,x306,x307,x308,x309,x310,x311
0,-0.132803,-0.149056,-0.094468,-0.191697,-0.114229,-0.114341,-0.096039,-0.115178,-0.157275,-0.116715,...,-0.103254,-0.122285,-0.149789,-0.127703,-0.094802,-0.119690,-0.141425,-0.123732,-0.081543,-0.227348
1,0.135038,0.076125,0.157854,0.030987,0.101192,0.111774,0.147497,0.173821,0.099808,0.115573,...,0.119518,0.197589,0.113135,0.136490,0.162921,0.099021,0.086653,0.158176,0.166968,0.048967


In [9]:
%%time
pd.DataFrame(dask.compute(
    minimum,
    maximum
), 
             columns=[f'x{i}' for i in range(arr.shape[1])])

CPU times: total: 1.47 s
Wall time: 843 ms


,x0,x1,x2,x3,x4,x5,x6,x7,x8,x9,...,x302,x303,x304,x305,x306,x307,x308,x309,x310,x311
0,-0.132803,-0.149056,-0.094468,-0.191697,-0.114229,-0.114341,-0.096039,-0.115178,-0.157275,-0.116715,...,-0.103254,-0.122285,-0.149789,-0.127703,-0.094802,-0.119690,-0.141425,-0.123732,-0.081543,-0.227348
1,0.135038,0.076125,0.157854,0.030987,0.101192,0.111774,0.147497,0.173821,0.099808,0.115573,...,0.119518,0.197589,0.113135,0.136490,0.162921,0.099021,0.086653,0.158176,0.166968,0.048967


3\. Найдите вектор $x \ne x_{256}$ из набора данных, ближайший к вектору $x_{256}$ в смысле метрики $L_1$. Выведите на экран первые 10 координат вектора $x$.

$$d_1(\textbf{x},\textbf{y})=\sum_{k=1}^{n}{|x_i - y_i|}, \textbf{x}, \textbf{y} \in \mathbb{R}^n$$

In [10]:
target = da.repeat(arr[:, 256], 311).reshape(-1,311)
vectors = da.delete(arr, 256, 1)

In [11]:
index = da.abs(vectors - target).sum(axis=1).argmin().compute()
arr[index, :10].compute()

array([ 0.02533725, -0.04231438,  0.02726739, -0.08028617,  0.0062374 ,
        0.00744976,  0.04478284,  0.02249521, -0.00062531, -0.05462844],
      dtype=float32)

4\. Рецепты разбиты на 4 группы. Загрузите маску для разбиения на группы из датасета `mask` из файла `recipe_embeddings.h5` в виде `dask.array`. Для каждой группы посчитайте и выведите на экран максимальное значение  нормы $\ell_1$ векторов рецептов, принадлежащих к этой группе. 

Подсказка: закодируйте маску принадлежности к группе при помощи метода кодирования one-hot encoding и воспользуйтесь механизмом распространения.

$$\ell_1: ||\textbf{x}||_1=\sum_{k=1}^{n}{|x_k|}, \textbf{x} \in \mathbb{R}^n$$

In [70]:
hdf = h5py.File(path+'recipe_embeddings.h5', 'r+')

dset = hdf['mask']

mask = da.from_array(dset)

norm = da.absolute(arr.sum(axis=1))
norm

dask.array<absolute, shape=(1200000,), dtype=float32, chunksize=(107546,), chunktype=numpy.ndarray>

In [50]:
mask_broadcasted = np.zeros((mask.size, mask.max() + 1))
mask_broadcasted[np.arange(mask.size), mask] = 1
mask_broadcasted = da.from_array(mask_broadcasted)
mask_broadcasted.compute()

array([[1., 0., 0., 0.],
       [1., 0., 0., 0.],
       [0., 1., 0., 0.],
       ...,
       [1., 0., 0., 0.],
       [0., 1., 0., 0.],
       [1., 0., 0., 0.]])

In [51]:
norm = da.repeat(norm, 4).reshape(-1, 4)

In [64]:
groups = (mask_broadcasted * norm)
groups.max(axis=0).compute()

array([0.89419359, 0.88820159, 0.89027524, 0.85577416])

5\. Пусть $X=[\textbf{x}_1,...\textbf{x}_M]^\top$ - матрица эмбеддингов рецептов размера $M\times N$, $W=[\textbf{w}_1,...,\textbf{w}_N]^\top$ - матрица коэффициентов некоторой модели машинного обучения размера $N\times 4$, $y=[y_1,...,y_M]^\top$ - вектор размера $M$, содержащий номера групп рецептов (метки классов). Тогда задачу классификации можно решить следующим образом: $$\hat{y_i} = argmax_j{<X_{i\cdot}, W_{\cdot j}>}$$ где $A_{i\cdot}$ обозначает $i$ строку матрицы, $A_{\cdot j}$ обозначает $j$ столбец матрицы, $\hat{y_i}$ - прогноз класса для рецепта $i$, $<\cdot, \cdot>$ - скалярное произведение векторов.

Инициализируйте матрицу $W$ случайным образом и получите прогнозы для всех рецептов при помощи этой матрицы и матрицы эмбеддингов. Подсчитайте и выведите на экран значение accuracy на основе полученных прогнозов $\hat{y}$ и правильных ответов $y$.

In [31]:
X = arr
W = da.random.random(size=(312, 4))
y = mask

dask.array<array, shape=(1200000,), dtype=int64, chunksize=(1200000,), chunktype=numpy.ndarray>

In [32]:
y_new = da.matmul(X, W).argmax(axis=1)

In [33]:
accuracy = (da.count_nonzero(y_new == y) / y.shape[0])
accuracy.compute()

0.3147975

6\. Сингулярным разложением (SVD) матрицы $A$ размера $M\times N$ называется разложение вида $A = USV^\top$, где $U$ - матрица размера $M\times N$  ортонормированных векторов произведения $AA^\top$, $V^T$ - транспонированная матрица размера $N\times N$ ортонормированных векторов произведения $A^\top A$, $S$ - диагональная матрица сингулярных значений размера $N\times N$.

SVD может быть использовано для понижения размерности векторов. Для этого от матрицы $U$ оставляют первые $k$ столбцов $U_{\cdot,:k}$, от матрицы $S$ оставляют левый верхний квадрат размера $k\times k$ $S_{:k,:k}$ и вычисляется произведение $\hat{A} = U_{\cdot,:k}S_{:k,:k}$

Выберите эмбеддинги тех рецептов, которые относятся к группе с номеров 3, и уменьшите их размерность до 64 при помощи реализации алгоритма SVD из пакета `dask.array.linalg`. Выведите количество строк и столбцов полученного массива.

Примечание: после отбора рецепта, принадлежащих третьей группе, вызовите у полученного массива метод `compute_chunk_sizes`, чтобы `dask` обновил метаинформацию в этом массиве. 

In [147]:
k=64
u, s, v = da.linalg.svd(arr[mask==3].compute_chunk_sizes())
u = u[:, :k]
s = da.diag(s)[:k, :k]
svd = da.matmul(u, s)
svd.shape

(10000, 64)

7\. Используя эмбеддинги уменьшенной размерности, полученные в задании 6, посчитайте косинусное сходство между каждой парой рецептов третьей группы. Выведите матрицу косинусного сходства на экран.

In [150]:
from sklearn.metrics.pairwise import cosine_similarity
cos_sim = da.array(cosine_similarity(svd))
print(cos_sim.compute())

[[1.0000002  0.77996165 0.70962805 ... 0.7856627  0.7249833  0.7074897 ]
 [0.77996165 1.         0.6422387  ... 0.8670994  0.6158872  0.65344274]
 [0.70962805 0.6422387  0.9999999  ... 0.6459563  0.8713087  0.87150264]
 ...
 [0.7856627  0.8670994  0.6459563  ... 1.         0.64296776 0.6637296 ]
 [0.7249833  0.6158872  0.8713087  ... 0.64296776 1.0000001  0.9126161 ]
 [0.7074897  0.65344274 0.87150264 ... 0.6637296  0.9126161  0.99999994]]


8\. Посчитайте и выведите на экран количество рецептов, для которых рецепт с индексом `242` входит число топ-5 ближайших рецептов в смысле косинусной близости. При поиске топ-5 рецептов для конкретного рецепта считайте, что он сам в это число не входит.

In [153]:
rec_242 = cos_sim[:, 242]
rec_242

dask.array<getitem, shape=(10000,), dtype=float32, chunksize=(5792,), chunktype=numpy.ndarray>

In [155]:
recs = da.delete(cos_sim, 256, 1)
recs 

dask.array<concatenate, shape=(10000, 9999), dtype=float32, chunksize=(5792, 5535), chunktype=numpy.ndarray>

9\. Графом называется совокупность двух множеств $G=(V,E)$: множества $V=\{v_1, ..., v_M\}$ узлов и множества ребер $E=\{(v_i, v_j)|v_i\in V, v_j\in V\}$, соединяющих эти узлы. Матрицей смежности невзвешенного графа называется квадратная матрица $A=[a_{ij}]$, в которой ${a_{ij}}$ обозначает количество ребер, соединяющих вершины $i$ и $j$.

Постройте матрицу смежности для графа рецептов на основе матрицы косинусного сходства между каждой парой рецептов. Будем считать, что между двумя рецептами в этом графе существует ребро, если косинусное сходство между двумя этими рецептами не менее 0.85. Петли (ребра из вершины в саму в себя) в графе должны отсутствовать. Посчитайте и выведите на экран количество ребер в данном графе. Проверьте, является ли полученная матрица смежности симметричной.

Примечание: считайте, что два различных рецепта не могут иметь косинусное сходство, равное 1.

In [160]:
matrix = da.where((cos_sim >= 0.85) & (cos_sim < 1), 1, 0)
if (matrix == da.transpose(matrix)).all().compute():
    print("Symmetrical")
else:
    print("Not symmetrical")

Symmetrical


10\. Работая с исходным файлом в формате `h5`, реализуйте алгоритм подсчета среднего вектора датасета в блочной форме.

Блочный алгоритм вычислений состоит из двух частей:
1. Загрузка фрагмента за фрагментом данных и проведение вычислений над этим фрагментом
2. Агрегация результатов вычислений на различных фрагментах для получения результата на уровне всего набора данных

Важно: при работе с `h5` в память загружаются не все элементы, а только те, которые запрашиваются в данный момент. При работе с `h5` вы можете работать с массивами `numpy.array`. Для итерации по сегментам файла допускается использование циклов.

Сравните время и результаты решения работы вашего алгоритма с реализацией поиска среднего вектора из `dask`. 

In [ ]:
with h5py.File(path+'demo.h5', 'r') as f:
    data = f['recipes']
    print(min(data))
    print(max(data))
    print(data[:15])

In [ ]:
import multiprocessing

def h5_avg_vec(file, chunks):
    lst_of_chunks = []
    with h5py.File(file,'r') as h5r:
            for chunk in range(chunks):
                